In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models, utils
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [5]:
print("Loading landmark dataset...")
data = pd.read_csv('asl_landmarks.csv')

# Separate features (the 63 numbers) from labels
X = data.drop('label', axis=1)
y = data['label']

print(f"Features shape: {X.shape}") # Should be (87000, 63)
print(f"Labels shape: {y.shape}")

Loading landmark dataset...
Features shape: (63676, 63)
Labels shape: (63676,)


In [6]:
print("Processing labels...")
# 1. Convert string labels ('A', 'B'...) to numbers (0, 1...)
label_encoder = LabelEncoder()
y_int = label_encoder.fit_transform(y)

# 2. One-hot encode the numbers (e.g., 2 -> [0, 0, 1, 0...])
num_classes = len(np.unique(y_int))
y_categorical = utils.to_categorical(y_int, num_classes=num_classes)

# 3. Create training and validation sets
X_train, X_test, y_train, y_test = train_test_split(
    X.values, # Convert DataFrame to numpy array
    y_categorical, 
    test_size=0.2, # 20% for testing
    random_state=42
)
print("Data ready.")

Processing labels...
Data ready.


In [7]:
model = models.Sequential([
    # Input layer expects a 1D vector of 63 numbers
    layers.Input(shape=(63,)),
    
    # "Brain" of the model
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(128, activation='relu'),
    
    # Output layer: 29 classes, 'softmax' for probabilities
    layers.Dense(num_classes, activation='softmax')
])

model.summary()

I0000 00:00:1762874112.799001  975288 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5561 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         8,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 29)             │         3,741 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 77,853 (304.11 KB)

 Trainable params: 77,853 (304.11 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy', # Use 'categorical' (not 'sparse')
    metrics=['accuracy']
)